<a href="https://colab.research.google.com/github/t-willi/Simula/blob/main/optimization_with_rayTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer 
import glob
import torch.optim as optim
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import os

import requests
import zipfile
from pathlib import Path
!pip install ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

if torch.cuda.is_available()==True:
  device="cuda:0"
else:
  device ="cpu"

(func pid=4866) 
 81%|████████  | 506/625 [06:20<01:08,  1.73it/s]
(func pid=5083) 
 54%|█████▎    | 671/1250 [05:19<05:17,  1.82it/s]
(func pid=4866) 
 81%|████████  | 507/625 [06:21<01:13,  1.61it/s]
(func pid=5083) 
 54%|█████▍    | 672/1250 [05:20<06:19,  1.52it/s]
(func pid=4866) 
 81%|████████▏ | 508/625 [06:21<01:20,  1.46it/s]
(func pid=5083) 
 54%|█████▍    | 673/1250 [05:21<06:41,  1.44it/s]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


(func pid=4866) 
 81%|████████▏ | 509/625 [06:23<01:40,  1.16it/s]
(func pid=5083) 
 54%|█████▍    | 674/1250 [05:22<07:10,  1.34it/s]
(func pid=4866) 
 82%|████████▏ | 510/625 [06:23<01:38,  1.17it/s]
(func pid=5083) 
 54%|█████▍    | 675/1250 [05:23<07:33,  1.27it/s]
(func pid=4866) 
 82%|████████▏ | 511/625 [06:24<01:37,  1.16it/s]
(func pid=5083) 
 54%|█████▍    | 676/1250 [05:24<08:22,  1.14it/s]
(func pid=4866) 
 82%|████████▏ | 512/625 [06:25<01:39,  1.14it/s]
(func pid=5083) 
 54%|█████▍    | 677/1250 [05:25<08:25,  1.13it/s]
(func pid=5083) 
 54%|█████▍    | 678/1250 [05:25<07:20,  1.30it/s]
(func pid=5083) 
 54%|█████▍    | 679/1250 [05:26<06:31,  1.46it/s]
(func pid=4866) 
 82%|████████▏ | 513/625 [06:27<02:08,  1.15s/it]
(func pid=5083) 
 54%|█████▍    | 680/1250 [05:26<06:18,  1.50it/s]
(func pid=5083) 
 54%|█████▍    | 681/1250 [05:27<06:19,  1.50it/s]
(func pid=4866) 
 82%|████████▏ | 514/625 [06:29<02:24,  1.30s/it]
(func pid=5083) 
 55%|█████▍    | 682/1250 [05:28<06:2

In [54]:
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [55]:
!pip install wandb -qq
import wandb
wandb.login()
run = wandb.init(project="Optimization_with_RayTune", job_type="optimization")


     |████████████████████████████████| 1.9 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 54.4 MB/s 
     |████████████████████████████████| 162 kB 53.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 162 kB 59.7 MB/s 
     |████████████████████████████████| 158 kB 67.9 MB/s 
     |████████████████████████████████| 157 kB 68.7 MB/s 
     |████████████████████████████████| 157 kB 58.6 MB/s 
     |████████████████████████████████| 157 kB 66.3 MB/s 
     |████████████████████████████████| 157 kB 65.3 MB/s 
     |████████████████████████████████| 157 kB 69.5 MB/s 
     |████████████████████████████████| 157 kB 76.7 MB/s 
     |████████████████████████████████| 157 kB 68.1 MB/s 
     |████████████████████████████████| 156 kB 64.6 MB/s 


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: twilli (ecg_simula). Use `wandb login --relogin` to force relogin


In [57]:
data = run.use_artifact('ecg_simula/setup_weights and biases/ecg_25000.zip:v0', type='raw_data')
data=data.download()

# Setup path to data folder
data_path = Path("data/")
train_path = data_path / "train_data"
# If the image folder doesn't exist, download it and prepare it... 
if train_path.is_dir():
    print(f"{train_path} directory exists.")
else:
    print(f"Did not find {train_path} directory, creating one...")
    train_path.mkdir(parents=True, exist_ok=True)

    # Unzip pizza, steak, sushi data
with zipfile.ZipFile("/content/artifacts/ecg_25000.zip:v0/ecg_25000.zip", "r") as zip_ref:
    print(f"Unzipping data to folder...") 
    zip_ref.extractall(train_path)

wandb: Downloading large artifact ecg_25000.zip:v0, 1383.92MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:20.5


Did not find data/train_data directory, creating one...
Unzipping data to folder...


In [58]:
# Create custom dataset class to load ECG data into dataset, containing 
# input tensor with lead 1 and output tensor with desired other leads
class Custom_dataset(Dataset):
    def __init__(self, data_dir,max_value=5011,column=3,split=False,target="train"):
      #get all files from directory loaded in all_files list
      self.column=column
      self.max_value=max_value
      #should shuffle the data here?
      self.files = glob.glob(data_dir + '/*.asc')
      self.len=len(self.files)
      self.cut1=int(self.len*0.8)
      self.cut2=int(self.len*0.9)
      self.train_files=self.files[0:self.cut1]
      self.test_files=self.files[self.cut1:self.cut2]
      self.val_files=self.files[self.cut2:self.len]
      self.target=target
      self.split=split

    def __len__(self):
      if self.split is True:
        if self.target is "train":
          return len(self.train_files)
        if self.target is "test":
          return len(self.test_files)
        if self.target is "val":
          return len(self.val_files)
      if self.split is not True:
        return len(self.files)

    def __getitem__(self,idx):
      header = [x for x in range(8)]
      #turn list of dataframes into Tensor
      if self.split is True:
        if self.target is "train":
          temp_df=pd.read_csv(self.train_files[idx],sep=" ", names = header)
        if self.target is "test":
          temp_df=pd.read_csv(self.test_files[idx],sep=" ", names = header)
        if self.target is "val":
          temp_df=pd.read_csv(self.val_files[idx],sep=" ", names = header)
      if self.split is not True:
        temp_df=pd.read_csv(self.files[idx],sep=" ", names = header)
      temp_df/=self.max_value
      #load input tensor
      
      temp_list_in=temp_df.iloc[:,0]
      #temp_list_in=normalize([temp_list_in], norm="max")
      temp_tensor_in = torch.tensor(temp_list_in,dtype=torch.float32)
      temp_tensor_in=temp_tensor_in.unsqueeze(0)
      #load label Tensor
      temp_list_out=temp_df.iloc[:,1:9].values
      #temp_list_out=normalize([temp_list_out], norm="max")
      temp_tensor_out=torch.tensor(temp_list_out,dtype=torch.float32)
      temp_tensor_out=temp_tensor_out.unsqueeze(0)
      temp_tensor_out=torch.permute(temp_tensor_out,(0,2,1))
      #combine input and label and output
      temp_tensor_pair= temp_tensor_in,temp_tensor_out
      return temp_tensor_pair

In [59]:
"""
Here, we define the autoencoder model.This model is taken from "https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py"
"""
class ECG_AE_v1(nn.Module):
    def __init__(self,step1=128,step2=64,step3=20):
        super(ECG_AE_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(5000,step1),
            nn.ReLU(),
            nn.Linear(step1,step2),
            nn.ReLU(),
            nn.Linear(step2,step3),
            nn.ReLU(),

        )
        self.decoder = nn.Sequential(

            nn.Linear(step3,step2),
            nn.ReLU(),
            nn.Linear(step2,step1),
            nn.ReLU(),
            nn.Linear(step1,35000),
        )


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = ECG_AE_v1().to(device)



In [53]:
data_dir="/content/data/train_data"

In [95]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
  Epochs=10
  net = ECG_AE_v1(config["step1"], config["step2"], config["step3"])
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

  if checkpoint_dir:
    model_state, optimizer_state = torch.load(
      os.path.join(checkpoint_dir, "checkpoint"))
    net.load_state_dict(model_state)
    optimizer.load_state_dict(optimizer_state)
  
  data_dir="/content/data/train_data"
  train_set = Custom_dataset(data_dir=data_dir,split=True,target="train")
  val_dataset = Custom_dataset(data_dir=data_dir,split=True,target="val")

  from torch.utils.data.dataloader import DataLoader
  BATCH_SIZE = int(config["batch_size"])
  #turn datasets into iterables
  train_dataloader = DataLoader(train_set,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                drop_last=True
                                )
  val_dataloader = DataLoader(val_dataset,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                              drop_last=True
                                )
  from tqdm.auto import tqdm
  #train and test loop
  for epoch in tqdm(range(Epochs)):
    print(f"Epoch:{epoch}")
    train_loss=0
    for batch, (X,y) in enumerate(tqdm(train_dataloader)):
      #print("doing train loop")
      X, y = X.to(device), y.to(device) 
      model.train()
      output=model(X)
      output=torch.reshape(output,(BATCH_SIZE, 1, 7, 5000))
      #print(output.shape,y.shape)
      loss = criterion(output,y)
      #print(loss)
      train_loss += loss
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #print(train_loss)
    #average loss per batch
    train_loss /= len(train_dataloader)
    #start testing
    val_loss = 0
    model.eval()
    with torch.inference_mode():
      for batch,(X,y) in enumerate(val_dataloader):
        #print("doing test loop")
        X, y = X.to(device), y.to(device)
        val_pred = model(X)
        val_pred=torch.reshape(val_pred,(BATCH_SIZE, 1, 7, 5000))
        val_loss += criterion(val_pred,y)   
      val_loss /= len(val_dataloader)

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save((net.state_dict(), optimizer.state_dict()), path)

      tune.report(loss=val_loss)

    print("Finished Training")


    


(func pid=8110) 
 24%|██▍       | 304/1250 [01:38<04:51,  3.25it/s]


In [64]:
def test_accuracy(net, device="cpu"):
  criterion = nn.MSELoss()
  test_dataset = Custom_dataset(data_dir=data_dir,split=True,target="test")
  testloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=4, shuffle=False)
  model.eval()
  with torch.inference_mode():
    for batch,(X,y) in enumerate(testloader):
      #print("doing test loop")
      X, y = X.to(device), y.to(device)
      test_pred = model(X)
      test_pred=torch.reshape(test_pred,(128, 1, 7, 5000))
      test_pred += criterion(test_pred,y)   
    val_loss /= len(testloader)
    return val_loss

In [67]:
config = {
    "step1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "step2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "step3": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([16,32,64,128,512])
}

In [68]:
from functools import partial

In [91]:
from ray.tune.trainable.session import checkpoint_dir
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
  data_dir = os.path.abspath("./data2")
  checkpoint_dir="/content/checkpoints"
  train_dir="/content/data/train_data"
  train_set = Custom_dataset(data_dir=train_dir,split=True,target="train")
  val_dataset = Custom_dataset(data_dir=train_dir,split=True,target="val")
  config = {
    "step1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "step2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "step3": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([16,32,64,128,512])
}
  scheduler = ASHAScheduler(
      metric="loss",
      mode="min",
      max_t=max_num_epochs,
      grace_period=1,
      reduction_factor=2)
  reporter = CLIReporter(
      # parameter_columns=["l1", "l2", "lr", "batch_size"],
      metric_columns=["loss","training_iteration"])
  result = tune.run(
      partial(train_cifar, data_dir=data_dir),
      resources_per_trial={"cpu": 1, "gpu": 1},
      config=config,
      num_samples=num_samples,
      scheduler=scheduler,
      progress_reporter=reporter)

  best_trial = result.get_best_trial("loss", "min", "last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["accuracy"]))

  best_trained_model = model(best_trial.config["step1"], best_trial.config["step2"], best_trial.config["step3"])
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if gpus_per_trial > 1:
          best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)

  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  test_acc = test_accuracy(best_trained_model, device)
  print("Best trial test set accuracy: {}".format(test_acc))



(func pid=7531) 
  5%|▌         | 68/1250 [00:34<09:06,  2.16it/s]


In [96]:
if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

(func pid=8110) 
 28%|██▊       | 354/1250 [01:54<04:44,  3.15it/s]
(func pid=8110) 
 28%|██▊       | 355/1250 [01:54<04:40,  3.19it/s]
(func pid=8110) 
 28%|██▊       | 356/1250 [01:54<05:12,  2.86it/s]
2022-10-18 17:29:54,939	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
(func pid=8110) 
 29%|██▊       | 357/1250 [01:55<05:34,  2.67it/s]
(func pid=8110) 
 29%|██▊       | 358/1250 [01:55<06:02,  2.46it/s]
(func pid=8110) 
 29%|██▊       | 359/1250 [01:56<06:16,  2.37it/s]
(func pid=8110) 
 29%|██▉       | 360/1250 [01:56<06:36,  2.25it/s]
(func pid=8110) 
 29%|██▉       | 361/1250 [01:57<06:50,  2.17it/s]
(func pid=8110) 
 29%|██▉       | 362/1250 [01:57<06:46,  2.18it/s]
(func pid=8110) 
 29%|██▉       | 363/1250 [01:58<06:52,  2.15it/s]
(func pid=8110) 
 29%|██▉       | 364/1250 [01

== Status ==
Current time: 2022-10-18 17:29:59 (running for 00:00:04.47)
Memory usage on this node: 2.3/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_cifar_2022-10-18_17-29-54
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------+
| Trial name              | status   | loc             |   batch_size |          lr |   step1 |   step2 |   step3 |
|-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------|
| train_cifar_7af34_00000 | RUNNING  | 172.28.0.2:9931 |          512 | 0.000177076 |      64 |      64 |       8 |
| train_cifar_7af34_00001 | PENDING  |                 |           32 | 0.0120769   |     128 |       4 |     

  0%|          | 0/39 [00:00<?, ?it/s]


(func pid=9931) Epoch:0
== Status ==
Current time: 2022-10-18 17:30:07 (running for 00:00:12.32)
Memory usage on this node: 6.2/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_cifar_2022-10-18_17-29-54
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------+
| Trial name              | status   | loc             |   batch_size |          lr |   step1 |   step2 |   step3 |
|-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------|
| train_cifar_7af34_00000 | RUNNING  | 172.28.0.2:9931 |          512 | 0.000177076 |      64 |      64 |       8 |
| train_cifar_7af34_00001 | PENDING  |                 |           32 | 0.0120769   | 

(func pid=9931) 
  3%|▎         | 1/39 [00:08<05:10,  8.17s/it]


...

== Status ==
Current time: 2022-10-18 17:38:35 (running for 00:08:40.70)
Memory usage on this node: 2.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -0.0018732730532065034 | Iter 1.000: -0.0028313915245234966
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_cifar_2022-10-18_17-29-54
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------+----------------+----------------------+
| Trial name              | status   | loc             |   batch_size |          lr |   step1 |   step2 |   step3 | loss           |   training_iteration |
|-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------+----------------+----------------------|
| train_cifar_7af34_00000 | RUNNING  | 172.28.0.2:9931 |          512 

2022-10-18 17:38:36,659	WARNING tune.py:687 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(func pid=9931) 
 56%|█████▋    | 22/39 [01:39<01:17,  4.55s/it]


== Status ==
Current time: 2022-10-18 17:38:40 (running for 00:08:45.72)
Memory usage on this node: 2.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -0.0018732730532065034 | Iter 1.000: -0.0028313915245234966
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_cifar_2022-10-18_17-29-54
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------+----------------+----------------------+
| Trial name              | status   | loc             |   batch_size |          lr |   step1 |   step2 |   step3 | loss           |   training_iteration |
|-------------------------+----------+-----------------+--------------+-------------+---------+---------+---------+----------------+----------------------|
| train_cifar_7af34_00000 | RUNNING  | 172.28.0.2:9931 |          512 

(func pid=9931) 
 59%|█████▉    | 23/39 [01:45<01:21,  5.07s/it]


KeyboardInterrupt: ignored